In [9]:
from google.cloud import aiplatform

PROJECT_ID = "titanic-problem"
REGION = "us-central1"
REPOSITORY = "titanic"
IMAGE = "xgboost-image"

aiplatform.init(project=PROJECT_ID, location=REGION)

import os

from google.cloud.aiplatform.prediction import LocalModel

from predictor.predictor import XgboostPredictorCPR  # Should be path of variable $USER_SRC_DIR

local_model = LocalModel.build_cpr_model(
    src_dir = "/home/jupyter/titanic/predictor",
    output_image_uri = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    predictor=XgboostPredictorCPR,
    requirements_path="/home/jupyter/titanic/predictor/requirements.txt"
)


In [4]:
!gcloud artifacts repositories create {REPOSITORY} --repository-format=docker \
--location=us-central1 --description="Docker repository"

ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


In [5]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [10]:
local_model.push_image()

In [48]:
f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}"

'us-central1-docker.pkg.dev/titanic-problem/titanic/xgboost-image'

In [11]:
import json

#input: ['pclass', 'name', 'sex', 'sibsp', 'parch']

sample = {"instances": [
    [1, 'Test Mr ABC', 'male', 2, 3],
    [1, "Test Ms PQR", "female", 0, 0],
    [2, "Test Mrs XYZ", "female", 0, 1],
]}

with open('instances.json', 'w') as fp:
    json.dump(sample, fp)


In [12]:
with local_model.deploy_to_local_endpoint(
    artifact_uri = '/home/jupyter/titanic/model_files/', # local path to artifacts
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file='instances.json',
        headers={"Content-Type": "application/json"},
    )
    print(predict_response)
    health_check_response = local_endpoint.run_health_check()


<Response [500]>


In [13]:
predict_response.content

b'{"detail":"The following exception has occurred: AttributeError. Arguments: (\\"\'super\' object has no attribute \'get_params\'\\",)."}'

In [3]:
predictor = XgboostPredictorCPR()

In [4]:
predictor.load('/home/jupyter/titanic/model_files/')

In [5]:
instances = {"instances":[[1, 'Test Mr ABC', 'male', 2, 3]]}
instances = predictor.preprocess(instances)

In [6]:
predictor.predict(instances)

array([0])

In [25]:
endpoint = local_model.deploy_to_local_endpoint(
    artifact_uri = '/home/jupyter/titanic/model_files', # local path to artifacts
)

In [22]:
endpoint.serve()

KeyboardInterrupt: 

In [21]:
endpoint.predict(
        request_file='instances.json',
        headers={"Content-Type": "application/json"},
    )

RuntimeError: The local endpoint is not serving traffic. Please call `serve()`.

In [40]:
import pandas as pd

In [31]:
instances = [[1, 'Test Mr ABC', 'male', 2, 3]]

In [41]:
def process_instance(data):
    pclass_map = {
            1: 1,
            2: 0,
            3: -1,
        }

    # input: ['pclass', 'name', 'sex', 'sibsp', 'parch']
    # output: ['pclass', 'cat_male', 'cat_Miss', 'cat_Mr', 'cat_Mrs', 'cat_Other', 'cat_big', 'cat_small']
    processed_instances = []
    for instance in instances:
        pclass = instance[0]
        name = instance[1]
        sex = instance[2]
        sibsp = instance[3]
        parch = instance[4]

        fam_size = sibsp + parch + 1
        if fam_size == 1:
            cat_big = 0
            cat_small = 0
        elif fam_size <= 4:
            cat_big = 0
            cat_small = 1
        else:
            cat_big = 1
            cat_small = 0

        if sex == "male":
            cat_male = 1
        else:
            cat_male = 0

        pclass = pclass_map[pclass]

        cat_Miss, cat_Mr, cat_Mrs, cat_Other = 0, 0, 0, 0
        if "Miss" in name:
            cat_Miss = 1
        elif "Mr" in name:
            cat_Mr = 1
        elif "Mrs" in name:
            cat_Mrs = 1
        elif "Master" in name:
            cat_Other = 0
        else:
            cat_Other = 1

        processed_instance = [
            pclass,
            cat_male,
            cat_Miss,
            cat_Mr,
            cat_Mrs,
            cat_Other,
            cat_big,
            cat_small,
        ]
        processed_instances.append(processed_instance)
    return pd.DataFrame(processed_instances, columns = ['pclass', 'cat_male', 'cat_Miss', 'cat_Mr', 'cat_Mrs', 'cat_Other', 'cat_big', 'cat_small'])

In [42]:
process_instance(data)

,pclass,cat_male,cat_Miss,cat_Mr,cat_Mrs,cat_Other,cat_big,cat_small
0,1,1,0,1,0,0,1,0


In [43]:
from xgboost import XGBClassifier
import xgboost as xgb
import pickle
xgb_model_loaded = pickle.load(open("model.pkl", "rb"))

In [44]:
xgb_model_loaded.predict(process_instance(data))

array([0])